In [ ]:
#All the required imports 
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
from glob import glob
from matplotlib import pyplot as plt
from PIL import Image as im
from array import *
import math
import numpy as np
import os
import cv2
from sklearn import preprocessing

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#train and test dir in google drive 
test_dir='../input/drive-retinal-vessel-segmentation-pixelwise/DRIVE_Modified/Modified_dataset_test'
train_dir='../input/drive-retinal-vessel-segmentation-pixelwise/DRIVE_Modified/Modified_dataset_train'

training_data = (glob(train_dir+'/img*.csv'))
testing_data = (glob(test_dir+'/img*.csv'))


In [ ]:
print('Number of training imgs= '+str(len(training_data)) +'\t'+'Number of testing imgs= '+str(len(testing_data)))

In [ ]:
#test and train file iterator 


final_training_data = pd.concat((pd.read_csv( file).assign(filename = file) 
                                for file in training_data), ignore_index = True)


final_testing_data = pd.concat((pd.read_csv(file).assign(filename = file) 
                                for file in testing_data), ignore_index = True)

In [ ]:
x_train=final_training_data[['R','G','B']].values
y_train =( final_training_data['truth labels'].values/255)
x_test=final_testing_data[['R','G','B']].values
y_test = (final_testing_data['truth labels'].values)/255



In [ ]:


x_train_scaled = x_train/255
x_test_scaled = x_test/255



#Oversampling of training data


In [ ]:
!pip install -U imbalanced-learn

In [ ]:
import imblearn
imblearn.__version__


In [ ]:
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE
adasyn = ADASYN()
smote=SMOTE()
x_train_scaled_oversampled, y_train = smote.fit_resample(x_train_scaled, y_train)




In [ ]:

test_file_img_pth='../input/drive-retinal-vessel-segmentation-pixelwise/DRIVE_Modified/Modified_dataset_test/img1.csv'
train_file_img_pth='../input/drive-retinal-vessel-segmentation-pixelwise/DRIVE_Modified/Modified_dataset_train/img1.csv'


test_img=pd.read_csv(test_file_img_pth)
train_img=pd.read_csv(train_file_img_pth)

test_img_1=test_img[['R','G','B']].values
test_img_1_scaled=test_img_1/255
train_img_1=train_img[['R','G','B']].values
train_img_1_scaled=train_img_1/255

test_img_1_label=(test_img['truth labels'].values/255)

train_img_1_label=(train_img['truth labels'].values)/255


In [ ]:
#oversampling on one img data
from imblearn.over_sampling import ADASYN
adasyn = ADASYN()
x_train_scaled_oversampled_1, y_train_1 = adasyn.fit_resample(train_img_1_scaled, train_img_1_label)

In [ ]:
from sklearn.neural_network import MLPClassifier

model=MLPClassifier(hidden_layer_sizes=(16,32,32,64), activation='relu', solver='adam',
                    alpha=0.001, batch_size=128, learning_rate= 'adaptive', 
                    learning_rate_init=0.001, max_iter=25, 
                    shuffle=True, random_state=420, verbose=2, 
                    warm_start=False,early_stopping=True, 
                    validation_fraction=0.1, beta_1=0.99, beta_2=0.9999, 
                    epsilon=1e-08, n_iter_no_change=5)


In [ ]:
model.fit(x_train_scaled_oversampled,y_train)

In [ ]:
print("Training set score: %f" % model.score(x_train_scaled_oversampled, y_train))
print("Test set score: %f" % model.score(x_test_scaled, y_test))

In [ ]:
y_train.shape

In [ ]:
new_final = []
result = model.predict(test_img_1_scaled)
# result=(result > 0.5)
# result

In [ ]:
plt.imshow(test_img['truth labels'].to_numpy().reshape((584,565),order='C'),cmap='gray')

In [ ]:
test_prediction = result.reshape((584,565),order='C')
plt.imshow(test_prediction,cmap='gray')

In [ ]:
result.sum()